# TFIDF - NMF analysis

In [1]:
import re
import time
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [2]:
# stopwords
with open('../data/euroleaks/stopwords.json', 'r') as f:
    stopwords = json.load(f)
    
# collocations
def apply_trigram_colloc(s, set_colloc):
    res = s.lower()
    for b1,b2,b3 in set_colloc:
        res = res.replace(f'{b1} {b2} {b3}', f'{b1}_{b2}_{b3}')
    return res

def apply_bigram_colloc(s, set_colloc):
    res = s.lower()
    for b1,b2 in set_colloc:
        res = res.replace(f'{b1} {b2}', f'{b1}_{b2}')
    return res

with open('../data/collocations/trigrams.json', 'r') as f:
    trigram_colloc = json.load(f)

with open('../data/collocations/bigrams.json', 'r') as f:
    bigram_colloc = json.load(f)

In [3]:
nlp = spacy.load("en_core_web_sm", exclude=["ner"])

## helper functions

In [4]:
def filter_token(token):
    return token.pos_ in {'ADJ', 'ADV', 'NOUN', 'PROPN', 'VERB'}\
        and not token.lemma_.lower() in nlp.Defaults.stop_words\
        and not token.lower_ in stopwords['names']\
        and not token.lower_ in stopwords['disfluency']\
        and not token.lemma_.lower() in stopwords['courtesy']\
        and len(token.lemma_) > 1

# Euroleaks

## document = speech

In [ ]:
leaks = pd.read_csv('../data/euroleaks/squeezed.csv')

# preprocess
leaks_documents = [
    ' '.join([token.lemma_.lower() for sentence in nlp(doc).sents for token in sentence
              if filter_token(token)
             ])
             for doc in leaks.speech.values
]

# leave out empty documents
leaks_documents = [d for d in leaks_documents if len(d)>1]

# apply collocations
leaks_documents = [
    apply_bigram_colloc(apply_trigram_colloc(doc, trigram_colloc), bigram_colloc)
    for doc in leaks_documents]

# tfidf
leaks_vectorizer = TfidfVectorizer(analyzer='word',
                                   min_df=10,
                                   max_df=0.75,
                                   smooth_idf=True,
                                   sublinear_tf=False)

leaks_X = leaks_vectorizer.fit_transform(leaks_documents)

- **TODO**: what are good min_df and max_df values? task-specific

In [ ]:
print(f'matrix shape: {leaks_X.shape}')

leaks_tfidf = leaks_X.sum(axis =0).A1
leaks_idf = leaks_vectorizer.idf_

plt.scatter(range(len(leaks_tfidf))[5:], np.sort(leaks_tfidf)[::-1][5:], marker='+')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(leaks_tfidf, leaks_idf, s=leaks_tfidf, edgecolors='black', label='tfidf')
#ax.legend()
ax.set_xlabel('tfidf')
ax.set_ylabel('idf')
ax.set_title('scatterplot of TFIDF and IDF values, scaled by TF')

# annotate words with highest tfidf
first_k = 10
sort_ix = np.argsort(leaks_tfidf)[::-1]
for ix in sort_ix[:first_k]:
    s = leaks_vectorizer.get_feature_names()[ix]
    jiggle_x = np.random.normal(0,0.1)
    jiggle_y = np.random.normal(0,0.1)
    ax.text(leaks_tfidf[ix]+jiggle_x, leaks_idf[ix]+jiggle_y, s)

In [ ]:
for word in np.array(leaks_vectorizer.get_feature_names())[sort_ix][:10]:
    print(word)

### NMF topic model
- https://shravan-kuchkula.github.io/nlp/topic-modeling/#build-nmf-model-using-sklearn

In [ ]:
# get topics with their terms and weights
def my_get_topics_terms_weights(weights, feature_names):
    feature_names = np.array(feature_names)
    sorted_indices = np.array([list(row[::-1]) for row in np.argsort(np.abs(weights))])
    sorted_weights = np.array([list(wt[index]) for wt, index in zip(weights, sorted_indices)])
    sorted_terms = np.array([list(feature_names[row]) for row in sorted_indices])

    topics = [{term: float(weight) for term, weight in zip(terms, term_weights)} for terms, term_weights in zip(sorted_terms, sorted_weights)]
    
    return topics

# get topics with their terms and weights
def get_topics_terms_weights(weights, feature_names):
    feature_names = np.array(feature_names)
    sorted_indices = np.array([list(row[::-1]) for row in np.argsort(np.abs(weights))])
    sorted_weights = np.array([list(wt[index]) for wt, index in zip(weights, sorted_indices)])
    sorted_terms = np.array([list(feature_names[row]) for row in sorted_indices])

    topics = [np.vstack((terms.T, term_weights.T)).T for terms, term_weights in zip(sorted_terms, sorted_weights)]

    return topics


# prints components of all the topics
# obtained from topic modeling
def print_topics_udf(topics, total_topics=1,
                     weight_threshold=0.0001,
                     display_weights=False,
                     num_terms=None):

    for index in range(total_topics):
        topic = topics[index]
        topic = [(term, float(wt))
                 for term, wt in topic]
        #print(topic)
        topic = [(word, round(wt,2))
                 for word, wt in topic
                 if abs(wt) >= weight_threshold]

        if display_weights:
            print('Topic #'+str(index)+' with weights')
            print(topic[:num_terms]) if num_terms else topic
        else:
            print('Topic #'+str(index)+' without weights')
            tw = [term for term, wt in topic]
            print(tw[:num_terms]) if num_terms else tw
        print()

# prints components of all the topics
# obtained from topic modeling
def get_topics_udf(topics, total_topics=1,
                     weight_threshold=0.0001,
                     num_terms=None):

    topic_terms = []

    for index in range(total_topics):
        topic = topics[index]
        topic = [(term, float(wt))
                 for term, wt in topic]
        #print(topic)
        topic = [(word, round(wt,2))
                 for word, wt in topic
                 if abs(wt) >= weight_threshold]

        topic_terms.append(topic[:num_terms] if num_terms else topic)

    return topic_terms

def getTermsAndSizes(topic_display_list_item):
    terms = []
    sizes = []
    for term, size in topic_display_list_item:
        terms.append(term)
        sizes.append(size)
    return terms, sizes

In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=3, init='nndsvd', alpha=0.1, l1_ratio=0.5)
nmf_output = nmf.fit_transform(leaks_X)

nmf_feature_names = leaks_vectorizer.get_feature_names()
nmf_weights = nmf.components_

In [ ]:
nmf_weights.shape

In [ ]:
topics = get_topics_terms_weights(nmf_weights, nmf_feature_names)

print_topics_udf(topics, total_topics=3, num_terms=20, display_weights=True)

In [ ]:
nmf_output[:5]

In [ ]:
leaks.speech[0]

## document = speaker

In [ ]:
leaks = pd.read_csv('../data/euroleaks/squeezed.csv')

# group by speaker
grouped = leaks.drop(columns=['date']).groupby('speaker').apply(lambda s: ' '.join(s.speech))
# get speaker labels
speakers = grouped.index

# make a list of all unidentified speakers
unidentified_speakers = [s for s in speakers if 'speaker' in s]
unidentified_speakers += [
    'irina',
    'kian',
    'male',
    'martin',
    'nabil',
    #'tooma', # I just know that he represents Finland
    'tropa'
]

# get identified speaker labels
identified_speakers = speakers[speakers.to_series().apply(lambda s: s not in unidentified_speakers)]
# filter out unidentified speakers
grouped = grouped[speakers.to_series().apply(lambda s: s not in unidentified_speakers)]

# preprocess
leaks_documents = [
    ' '.join([token.lemma_.lower() for sentence in nlp(doc).sents for token in sentence
              if filter_token(token)
             ])
             for doc in grouped.values
]

# leave out empty documents
leaks_documents = [d for d in leaks_documents if len(d)>1]

print(f'There are {len(leaks_documents)} documents.')

# apply collocations
leaks_documents = [
    apply_bigram_colloc(apply_trigram_colloc(doc, trigram_colloc), bigram_colloc)
    for doc in leaks_documents]

# need to filter out unidentified speakers first?

# tfidf
leaks_vectorizer = TfidfVectorizer(analyzer='word',
                                   min_df=2,
                                   max_df=0.75,
                                   smooth_idf=True,
                                   sublinear_tf=False)

leaks_X = leaks_vectorizer.fit_transform(leaks_documents)

In [ ]:
int(31 * 0.75)

In [ ]:
# speakers that have less than cutoff tokens after preprocessing
cutoff = 100
speakers[speakers.to_series().apply(lambda s: s not in unidentified_speakers)][np.array([len(d.split()) for d in leaks_documents]) < cutoff]

What are good min_df and max_df values? Need to choose them specifically for the task at hand; meaning clustering speakers.

Therefore, I want to remove words which have low discriminatory power between speaker.

In [ ]:
print(f'matrix shape: {leaks_X.shape}')

leaks_tfidf = leaks_X.sum(axis =0).A1
leaks_idf = leaks_vectorizer.idf_

plt.scatter(range(len(leaks_tfidf))[5:], np.sort(leaks_tfidf)[::-1][5:], marker='+')

In [ ]:
leaks_tfidf = leaks_X.sum(axis =0).A1
sort_ix = np.argsort(leaks_tfidf)[::-1]

for word in np.array(leaks_vectorizer.get_feature_names())[sort_ix][:10]:
    print(word)

### NMF

#### cross validation to find number of latent dimenstions

In [ ]:
# adapted from:
# https://gist.github.com/ahwillia/65d8f87fcd4bded3676d67b55c1a3954
# nonnegfac: https://github.com/kimjingu/nonnegfac-python

import numpy as np
from numpy.random import randn, rand
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from nonnegfac.nnls import nnlsm_blockpivot as nnlstsq
import itertools
from scipy.spatial.distance import cdist


def censored_nnlstsq(A, B, M):
    """Solves nonnegative least-squares problem with missing data in B
    Args
    ----
    A (ndarray) : m x r matrix
    B (ndarray) : m x n matrix
    M (ndarray) : m x n binary matrix (zeros indicate missing values)
    Returns
    -------
    X (ndarray) : nonnegative r x n matrix that minimizes norm(M*(AX - B))
    """
    
    if A.ndim == 1:
        A = A[:,None]
    rhs = np.dot(A.T, M * B).T[:,:,None] # n x r x 1 tensor # dimension mismatch here
    T = np.matmul(A.T[None,:,:], M.T[:,:,None] * A[None,:,:]) # n x r x r tensor
    X = np.empty((B.shape[1], A.shape[1]))
    for n in range(B.shape[1]):
        X[n] = nnlstsq(T[n], rhs[n], is_input_prod=True)[0].T
    return X.T


def cv_nmf(data, rank, M=None, p_holdout=0.3):
    """Fit NMF while holding out a fraction of the dataset.
    """

    solver = censored_nnlstsq

    # create masking matrix
    if M is None:
        M = np.random.rand(*data.shape) > p_holdout

    # initialize U randomly
    U = np.random.rand(data.shape[0], rank)

    # fit nmf
    for itr in range(50):
        Vt = solver(U, data, M)
        U = solver(Vt.T, data.T, M.T).T
    
    # return result and test/train error
    resid = np.dot(U, Vt) - data
    train_err = np.mean(resid[M]**2)
    test_err = np.mean(resid[~M]**2)
    return U, Vt, train_err, test_err


def validate_nmf(data, M=None):
    
    # parameters
    replicates = 1
    ranks = np.arange(1, 4)
    
    train_err, test_err = [], []

    # fit models
    for rnk, _ in itertools.product(ranks, range(replicates)):
        print(rnk)
        tr, te = cv_nmf(data, rnk, M)[2:]
        train_err.append((rnk, tr))
        test_err.append((rnk, te))
        
    return train_err, test_err

In [ ]:
def eye_mask(shape):
    return np.vstack([np.eye(shape[1]) for _ in range(int(shape[0] / shape[1])+1)])[:shape[0],:].astype(int)

In [ ]:
# control with 6 latent dimensions
# control_n_latent_dim = 6
# M, N = 100, 150
# noise = .8

# U = np.random.rand(M, control_n_latent_dim)
# Vt = np.random.rand(control_n_latent_dim, N)
# control_X = np.dot(U, Vt) + noise*np.random.rand(M, N)

# control_train_err, control_test_err = validate_nmf(control_X)

In [ ]:
# try to construct a mask that takes sparseness into account, but often singular, no time to go into this

# p_holdout = 0.3

# nonzero = np.where(leaks_X.A > 0)

# mask_ = np.random.rand(nonzero[0].size) > p_holdout

# nonzero = (nonzero[0][mask_], nonzero[1][mask_])

# mask = np.ones(leaks_X.shape).astype(int)
# mask[nonzero] = 0

In [ ]:
# validate tfidf matrix
#leaks_train_err, leaks_test_err = validate_nmf(leaks_X.A, mask)

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10,4))

# titles = [f'Control ({control_n_latent_dim} latent dimensions)', 'Euroleaks identified speakers']
# train_errs = [control_train_err, leaks_train_err]
# test_errs = [control_test_err, leaks_test_err]

# for i, ax in enumerate(axes):
    
#     ax.plot(*list(zip(*train_errs[i])), '-xb', label='train')
#     ax.plot(*list(zip(*test_errs[i])), '-or', label='test')
#     ax.set_title(titles[i])
#     #ax.legend()
        
#     ax.set_xlabel('Number of factors')
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
    
#     if i == 0:
#         ax.set_ylabel('Mean Squared Error')
#         ax.axvline(control_n_latent_dim, color='k', dashes=[2,2])
#     else:
#         ax.legend()
    
# # lines, labels = axes[0].get_legend_handles_labels()
# # fig.legend(lines, labels, loc='upper right')
# # fig.subplots_adjust(right=0.9)
    
# fig.tight_layout()
# #fig.savefig('../figures/nmf-speaker-validation.png')

In [ ]:
print(f'Percent of entries in X which are 0: {np.round(np.mean(leaks_X.A == 0)*100,2)} %')

#### 3D plot

In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=3,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          random_state=0, # important sicne otherwise the resulting plot will be different each run
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(leaks_X.toarray())
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
# get speaker labels (name + entity they represent)
with open('../data/euroleaks/name_to_entity.json', 'r') as f:
    speaker_to_entity = json.load(f)

institutions = ['ECB', 'IMF', 'European Commission', 'ESM', 'EFC']
markers = ['\u2020', '\u2021']

def intervention(s):
    if s not in speaker_to_entity.keys():
        return 'Unidentified'
    if speaker_to_entity[s] == 'European Commission':
        return 'EC'
    return speaker_to_entity[s]

def make_label_from_speaker(s):
    return f'{markers[0] if speaker_to_entity[s] in institutions else markers[1]} {s.title()} ({intervention(s)})'

labels = identified_speakers.to_series().apply(lambda s: make_label_from_speaker(s)).to_numpy()

In [ ]:
# correct text annotations so that they don't overlap

offset = [
    (.01,.0,.025), # Alexander Stubb
    (.01,.01,.01), # Benoît Cœuré
    (.01,-.08,-.01), # Christine Lagarde 
    (-.04,.07,-.16), # Declan Costello
    (.01,.01,.01), # Dušan Mramor
    (-.15,-.18,.03), # Edward Scicluna  
    (.01,.01,.01), # Hans Jörg Schelling  
    (.015,.0,-.02), # Harris Georgiades
    (.02,-.005,.01), # Jeroen Dijsselbloem
    (-.23,-.18,-.03), # Johan Van Overtveldt
    (-.15,-.1,-.03), # Jānis Reirs
    (.015,.005,.0), # Klaus Regling
    (-.01,-.02,-.06), # Luca Antonio Ricci
    (.02,-.05,.0), # Luis De Guindos
    (-.13,-.08,-.05), # Luis Pierre
    (-.13,-.08,.005), # Marco Buti
    (.01,.01,.01), # Maria Luís Albuquerque
    (.01,.01,.01), # Mario Draghi
    (-.17,-.12,-.1), # Michael Noonan
    (-.16,-.12,-.05), # Michel Sapin
    (-.23,-.13,-.06), # Nikos Theocarakis
    (-.15,-.18,.005), # Peter Kažimír
    (.02,-.01,.0), # Pier Carlo Padoan
    (.01,.01,.0), # Pierre Moscovici
    (.02,.0,.0), # Poul Mathias Thomsen
    (.01,.015,.005), # Rimantas Šadžius
    (.01,.01,.01), # Thomas Steffen
    (.01,.015,-.01), # Thomas Wieser
    (-.12,-.12,.0), # Tooma
    (.01,.01,.0), # Wolfgang Schäuble
    (.01,.01,.0), # Yanis Varoufakis
]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

# remove axis ticks
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_zlabel('z')

for counter, (point, label) in enumerate(zip(W, labels)):

    x,y,z = point
    ax.scatter( x, y, z,
                color=(x,y,z),
                edgecolor=None,
                alpha=0.3,
                s=100)
    
    ax.text(x + offset[counter][0],
            y + offset[counter][1],
            z + offset[counter][2],
            label,
            zdir=(1,1,0))

#ax.axis('off')

#fig.savefig('../figures/3d-speakers-leaks.png')

#### clustering
Each axis captures the base topic of a particular document cluster, and each document is represented as an additive combination of the base topics. The cluster membership of each document can be easily determined by finding the base topic (the axis) with which the document has the largest projection value.

In [ ]:
print(f'There are {len(identified_speakers)} speakers.')

In [ ]:
number_of_clusters = 3

nmf = NMF(n_components=number_of_clusters,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(leaks_X.toarray())
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
from collections import defaultdict

clusters = defaultdict(list)

for speaker, cluster in  zip(identified_speakers, W.argmax(axis=1)):
    clusters[cluster].append(speaker)

In [ ]:
for key,values in clusters.items():
    print(f'cluster {key}:')
    for v in values:
        print(f'\t{v}')
    print()

In [ ]:
topics = get_topics_terms_weights(H, leaks_vectorizer.get_feature_names())

print_topics_udf(topics, total_topics=3, num_terms=15, display_weights=True)

In [ ]:
def make_label_from_speaker(s):
    return f'{markers[0] if speaker_to_entity[s] in institutions else markers[1]} {s.title()}\\newline({speaker_to_entity[s] if s in speaker_to_entity.keys() else "Unkown"})'


def format_latex_table(first_n, topics, clusters):

    header = 'topic 0 & topic 1 & topic 2 \\\\'

    print('\hline')
    print(header)
    print('\hline')
    
    for j in range(max([len(c) for c in clusters.values()])):
        
        #print('\\centering ', end='')
        
        for i in range(len(topics)):

            if j < len(clusters[i]):
                print(f'{make_label_from_speaker(clusters[i][j])} ', end='')

            if i < len(topics)-1:
                print('& ', end='')
                
        print('\\\\')
        
    print('\hline')

    for j in range(first_n):
        
        #print('\\centering ', end='')
        
        for i in range(len(topics)):

            word = topics[i][j,0].replace("_", "\\_")
            if i == len(topics) - 1:
                print(f'{word} \\\\')
            else:
                print(f'{word} & ', end='')

    print('\hline')

In [ ]:
format_latex_table(10, topics, clusters)

## document = date

In [ ]:
leaks = pd.read_csv('../data/euroleaks/squeezed.csv')

# group by speaker
grouped = leaks.drop(columns=['speaker']).groupby('date').apply(lambda s: ' '.join(s.speech))
# get speaker labels
dates = grouped.index

# preprocess
leaks_documents = [
    ' '.join([token.lemma_.lower() for sentence in nlp(doc).sents for token in sentence
              if filter_token(token)
             ])
             for doc in grouped.values
]

# leave out empty documents
leaks_documents = [d for d in leaks_documents if len(d)>1]

print(f'There are {len(leaks_documents)} documents.')

# apply collocations
leaks_documents = [
    apply_bigram_colloc(apply_trigram_colloc(doc, trigram_colloc), bigram_colloc)
    for doc in leaks_documents]

# need to filter out unidentified speakers first?

# tfidf
leaks_vectorizer = TfidfVectorizer(analyzer='word',
                                   min_df=2,
                                   max_df=0.75,
                                   smooth_idf=True,
                                   sublinear_tf=False)

leaks_X = leaks_vectorizer.fit_transform(leaks_documents)

In [ ]:
int(12*0.75)

In [ ]:
leaks_X.shape

### cross validation to find number of latent dimenstions

In [ ]:
# control with 6 latent dimensions
# control_n_latent_dim = 3
# M, N = 100, 150
# noise = .8

# U = np.random.rand(M, control_n_latent_dim)
# Vt = np.random.rand(control_n_latent_dim, N)
# control_X = np.dot(U, Vt) + noise*np.random.rand(M, N)

# control_train_err, control_test_err = validate_nmf(control_X)

In [ ]:
# validate tfidf matrix
#leaks_train_err, leaks_test_err = validate_nmf(leaks_X.A)

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10,4))

# titles = [f'Control ({control_n_latent_dim} latent dimensions)', 'Euroleaks identified speakers']
# train_errs = [control_train_err, leaks_train_err]
# test_errs = [control_test_err, leaks_test_err]

# for i, ax in enumerate(axes):
    
#     ax.plot(*list(zip(*train_errs[i])), '-xb', label='train')
#     ax.plot(*list(zip(*test_errs[i])), '-or', label='test')
#     ax.set_title(titles[i])
#     #ax.legend()
        
#     ax.set_xlabel('Number of factors')
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
    
#     if i == 0:
#         ax.set_ylabel('Mean Squared Error')
#         ax.axvline(control_n_latent_dim, color='k', dashes=[2,2])
    
# lines, labels = axes[0].get_legend_handles_labels()
# fig.legend(lines, labels, loc='upper right')
# fig.subplots_adjust(right=0.9)
    
# #fig.tight_layout()
# fig.savefig('../figures/nmf-date-validation.png')

In [ ]:
print(f'Percent of entries in X which are 0: {np.round(np.mean(leaks_X.A > 0)*100,2)} %')

### 3D plot

In [ ]:
nmf = NMF(n_components=3,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          random_state=0, # important sicne otherwise the resulting plot will be different each run
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(leaks_X.toarray())
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
# from: https://stackoverflow.com/questions/22867620/putting-arrowheads-on-vectors-in-matplotlibs-3d-plot

from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [ ]:
offset = [
    (-.055,-.055,.0),
    (-.055,-.055,.0),
    (.01,.01,.01),
    (.01,.01,.01),
    (.01,.01,.01),
    (-.055,-.055,-.01),
    (-.055,-.055,-.01),
    (-.055,-.055,.0),
    (.01,.01,.01),
    
    (-.055,-.085,-.01),
    (.04,-.04,.01),
    (.01,.01,.01),
]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

# remove axis ticks
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_zlabel('z')

for i, label in enumerate(dates):

    point = W[i,:]
    
    # plot
    x,y,z = point
    c = (min(max(x,0),1),min(max(y,0),1),min(max(z,0),1))
    ax.scatter( x, y, z,
                color=c,
                edgecolor=None,
                alpha=0.5,
                s=100)
    
    # annotate
    ax.text(x + offset[i][0],
            y + offset[i][1],
            z + offset[i][2],
            pd.to_datetime(label).strftime('%d/%m'),
            zdir=(1,1,0))
    
    # add arrow
    if i>0:
        previous_point = W[i-1,:]
        px, py, pz = previous_point
        #ax.arrow((px,x),(py,y),(pz,z),
        #        color='black',
        #        arrowstyle='-|>')
        #ax.quiver(px,py,pz,
        #          x,y,z,
        #          color='black',
        #          alpha=0.3,
        #          lw=2)
        arrow = Arrow3D([px,x],[py,y],[pz,z],
                        mutation_scale=20,
                        lw=1,
                        arrowstyle='-|>',
                        color='k',
                        alpha=0.3)
        ax.add_artist(arrow)

fig.savefig('../figures/3d-dates-leaks.png')

### clustering
Each axis captures the base topic of a particular document cluster, and each document is represented as an additive combination of the base topics. The cluster membership of each document can be easily determined by finding the base topic (the axis) with which the document has the largest projection value.

In [ ]:
print(f'There are {len(dates)} dates.')

In [ ]:
number_of_clusters = 3

nmf = NMF(n_components=number_of_clusters,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(leaks_X.toarray())
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
from collections import defaultdict

clusters = defaultdict(list)

for date, cluster in  zip(dates, W.argmax(axis=1)):
    clusters[cluster].append(pd.to_datetime(date).strftime('%d/%m'))

In [ ]:
for key,values in clusters.items():
    print(f'cluster {key}:')
    for v in values:
        print(f'\t{v}')
    print()

In [ ]:
topics = get_topics_terms_weights(H, leaks_vectorizer.get_feature_names())

print_topics_udf(topics, total_topics=3, num_terms=15, display_weights=True)

In [ ]:
[ len(c) for c in clusters.values() ]

In [ ]:
def format_latex_table(first_n, topics, clusters):

    header = '\multicolumn{2}{|c||}{topic 0} & \multicolumn{2}{|c||}{topic 1} & \multicolumn{2}{|c|}{topic 2} \\\\'

    print('\hline')
    print(header)
    print('\hline')

    for j in range(first_n):

        for i in range(len(topics)):

            if j < len(clusters[i]):
                print(f'{clusters[i][j]} & ', end='')
            else:
                print('& ', end='')

            word = topics[i][j,0].replace("_", "\\_")
            if i == len(topics) - 1:
                print(f'{word} \\\\')
            else:
                print(f'{word} & ', end='')

    print('\hline')

In [ ]:
format_latex_table(15, topics, clusters)

# Communiques

In [ ]:
communiques = pd.read_csv('../data/communiques/cleaned.csv')

# group by speaker
grouped = communiques.drop(columns=['title']).groupby('date').apply(lambda s: ' '.join(s.story))
# get speaker labels
dates = grouped.index

# preprocess
comm_documents = [
    ' '.join([token.lemma_.lower() for sentence in nlp(doc).sents for token in sentence
              if filter_token(token)
             ])
             for doc in grouped.values
]

# leave out empty documents
comm_documents = [d for d in comm_documents if len(d)>1]

print(f'There are {len(comm_documents)} documents.')

# apply collocations
comm_documents = [
    apply_bigram_colloc(apply_trigram_colloc(doc, trigram_colloc), bigram_colloc)
    for doc in comm_documents]

# need to filter out unidentified speakers first?

# tfidf
comm_vectorizer = TfidfVectorizer(analyzer='word',
                                   min_df=2,
                                   max_df=0.75,
                                   smooth_idf=True,
                                   sublinear_tf=False)

comm_X = comm_vectorizer.fit_transform(comm_documents)

In [ ]:
comm_X.shape

### cross validation to find number of latent dimenstions

In [ ]:
# control with 6 latent dimensions
# control_n_latent_dim = 3
# M, N = 100, 150
# noise = .8

# U = np.random.rand(M, control_n_latent_dim)
# Vt = np.random.rand(control_n_latent_dim, N)
# control_X = np.dot(U, Vt) + noise*np.random.rand(M, N)

# control_train_err, control_test_err = validate_nmf(control_X)

In [ ]:
# validate tfidf matrix
#comm_train_err, comm_test_err = validate_nmf(comm_X.A)

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10,4))

# titles = [f'Control ({control_n_latent_dim} latent dimensions)', 'Communiques dates']
# train_errs = [control_train_err, comm_train_err]
# test_errs = [control_test_err, comm_test_err]

# for i, ax in enumerate(axes):
    
#     ax.plot(*list(zip(*train_errs[i])), '-xb', label='train')
#     ax.plot(*list(zip(*test_errs[i])), '-or', label='test')
#     ax.set_title(titles[i])
#     #ax.legend()
        
#     ax.set_xlabel('Number of factors')
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
    
#     if i == 0:
#         ax.set_ylabel('Mean Squared Error')
#         ax.axvline(control_n_latent_dim, color='k', dashes=[2,2])
    
# lines, labels = axes[0].get_legend_handles_labels()
# fig.legend(lines, labels, loc='upper right')
# fig.subplots_adjust(right=0.9)
    
# #fig.tight_layout()
# fig.savefig('../figures/nmf-date-comm-validation.pdf')

In [ ]:
print(f'Percent of entries in X which are 0: {np.round(np.mean(comm_X.A > 0)*100,2)} %')

### 3D plot

In [ ]:
nmf = NMF(n_components=3,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          random_state=0,
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(comm_X.A)
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
# from: https://stackoverflow.com/questions/22867620/putting-arrowheads-on-vectors-in-matplotlibs-3d-plot

from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [ ]:
pd.to_datetime(dates).strftime('%d/%m')

In [ ]:
offset = [
    (-.055,-.055,.0),
    (.01,.01,.01),
    (.01,.01,.01),
    (.01,.01,.01),
    (-.055,-.055,.0),
    (.01,.01,.01),
    (.01,.01,.01),
    (-.02,-.03,.045),
    (-.05,-.055,-.05),
    (.03,-.1,.0),
    (.01,.01,.01),
    (.01,.01,.01),
    (-.055,-.055,.0),
]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

# remove axis ticks
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_zlabel('z')

for i, label in enumerate(dates):

    point = W[i,:]
    
    # plot
    x,y,z = point
    c = (min(max(x,0),1),min(max(y,0),1),min(max(z,0),1))
    ax.scatter( x, y, z,
                color=c,
                edgecolor=None,
                alpha=0.5,
                s=100)
    
    # annotate
    ax.text(x + offset[i][0],
            y + offset[i][1],
            z + offset[i][2],
            pd.to_datetime(label).strftime('%d/%m'),
            zdir=(1,1,0))
    
    # add arrow
    if i>0:
        previous_point = W[i-1,:]
        px, py, pz = previous_point
        #ax.arrow((px,x),(py,y),(pz,z),
        #        color='black',
        #        arrowstyle='-|>')
        #ax.quiver(px,py,pz,
        #          x,y,z,
        #          color='black',
        #          alpha=0.3,
        #          lw=2)
        arrow = Arrow3D([px,x],[py,y],[pz,z],
                        mutation_scale=20,
                        lw=1,
                        arrowstyle='-|>',
                        color='k',
                        alpha=0.3)
        ax.add_artist(arrow)
        
fig.savefig('../figures/3d-dates-comms.png')

### clustering
Each axis captures the base topic of a particular document cluster, and each document is represented as an additive combination of the base topics. The cluster membership of each document can be easily determined by finding the base topic (the axis) with which the document has the largest projection value.

In [ ]:
print(f'There are {len(dates)} dates.')

In [ ]:
number_of_clusters = 3

nmf = NMF(n_components=number_of_clusters,
          init='nndsvd',
          beta_loss='frobenius',
          max_iter=1000,
          alpha=0,
          l1_ratio=0)

W = nmf.fit_transform(comm_X.toarray())
H = nmf.components_

print(f'W shape: {W.shape}')
print(f'H shape: {H.shape}')

In [ ]:
from collections import defaultdict

clusters = defaultdict(list)

for date, cluster in  zip(dates, W.argmax(axis=1)):
    clusters[cluster].append(pd.to_datetime(date).strftime('%d/%m'))

In [ ]:
for key,values in clusters.items():
    print(f'cluster {key}:')
    for v in values:
        print(f'\t{v}')
    print()

In [ ]:
topics = get_topics_terms_weights(H, comm_vectorizer.get_feature_names())

print_topics_udf(topics, total_topics=3, num_terms=15, display_weights=True)

In [ ]:
format_latex_table(10, topics, clusters)